In [48]:
#N gram with Pytorch
import torch 
from torch import nn 


In [49]:
import torch.nn as nn

class MyModel(nn.Module):
    def __init__(self):
        super(MyModel, self).__init__()
        self.linear1 = nn.Linear(in_features, out_features)  # Define the linear layer

    def forward(self, x):
        return self.linear1(x)  # Apply the linear layer in the forward pass

In [50]:
embedding_dim=3
context_size=2
vocab_size=4

embeddings = nn.Embedding(vocab_size, embedding_dim)
linear = nn.Linear(context_size * embedding_dim, 2)

In [51]:
inputs = torch.tensor([2,3])

In [52]:
my_embedding = embeddings(inputs)

In [53]:
my_embedding_c = torch.reshape( my_embedding, (-1, context_size * embedding_dim))

In [54]:
linear(my_embedding_c)

tensor([[0.0150, 0.3244]], grad_fn=<AddmmBackward0>)

In [55]:
import torch
import torch.nn as nn
import torch.nn.functional as F  # <-- ADD THIS

class NgramLanguageModeler(nn.Module):
    def __init__(self, vocab_size, embedding_dim, context_size):
        super(NgramLanguageModeler, self).__init__()

        self.context_size = context_size
        self.embedding_dim = embedding_dim

        self.embeddings = nn.Embedding(vocab_size, embedding_dim)
        self.linear1 = nn.Linear(context_size * embedding_dim, 128)
        self.linear2 = nn.Linear(128, vocab_size)

    def forward(self, inputs):
        embeds = self.embeddings(inputs)
        embeds = torch.reshape(embeds, (-1, self.context_size * self.embedding_dim))

        out = F.relu(self.linear1(embeds))  # <-- FIXED LAYER NAME
        out = self.linear2(out)

        return out


In [56]:
CONTEXT_SIZE = 2
BATCH_SIZE = 10 
EMBEDDING_DIM = 10

def collate_batch(batch):
    batch_size = len(batch)
    context, target = [], []

    for i in range(CONTEXT_SIZE, batch_size):
        target.append(vocab([batch[i]]))
        context.append(vocab([batch[i-j-1] for j in range(CONTEXT_SIZE)]))
        return torch.tensor(context).to(device), torch.tensor(target).to(device.reshape(-1))

In [57]:
from torchtext.data.utils import get_tokenizer

# Choose tokenizer
tokenizer = get_tokenizer("basic_english")

# Text to tokenize
song = (
    "Never gonna break your loop ml parody "
    "to the tune of never gonna give you up "
    "we are no strangers to code "
    "you train the weights and I watch it explode "
    "backprop's running through the night "
    "tuning those layers 'til the loss feels right "
    "and if you ask me how it works "
    "don’t you worry I’ll just say "
    "it’s gradients and matrix math "
    "with tensors dancing every way "
    "never gonna break your loop "
    "never gonna drop your batch "
    "never gonna run out of RAM and crash you "
    "never gonna overfit "
    "never gonna undershoot "
    "never gonna tell a model lie and bash you"
)

# Apply tokenizer to the song
tokens = tokenizer(song)

print(tokens)


['never', 'gonna', 'break', 'your', 'loop', 'ml', 'parody', 'to', 'the', 'tune', 'of', 'never', 'gonna', 'give', 'you', 'up', 'we', 'are', 'no', 'strangers', 'to', 'code', 'you', 'train', 'the', 'weights', 'and', 'i', 'watch', 'it', 'explode', 'backprop', "'", 's', 'running', 'through', 'the', 'night', 'tuning', 'those', 'layers', "'", 'til', 'the', 'loss', 'feels', 'right', 'and', 'if', 'you', 'ask', 'me', 'how', 'it', 'works', 'don’t', 'you', 'worry', 'i’ll', 'just', 'say', 'it’s', 'gradients', 'and', 'matrix', 'math', 'with', 'tensors', 'dancing', 'every', 'way', 'never', 'gonna', 'break', 'your', 'loop', 'never', 'gonna', 'drop', 'your', 'batch', 'never', 'gonna', 'run', 'out', 'of', 'ram', 'and', 'crash', 'you', 'never', 'gonna', 'overfit', 'never', 'gonna', 'undershoot', 'never', 'gonna', 'tell', 'a', 'model', 'lie', 'and', 'bash', 'you']


In [58]:
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator

tokenizer = get_tokenizer("basic_english")

# Song is a single long string — this is fine
song = (
    "Never gonna break your loop ml parody "
    "to the tune of never gonna give you up "
    "we are no strangers to code "
    "you train the weights and I watch it explode "
    "backprop's running through the night "
    "tuning those layers 'til the loss feels right "
    "and if you ask me how it works "
    "don’t you worry I’ll just say "
    "it’s gradients and matrix math "
    "with tensors dancing every way "
    "never gonna break your loop "
    "never gonna drop your batch "
    "never gonna run out of RAM and crash you "
    "never gonna overfit "
    "never gonna undershoot "
    "never gonna tell a model lie and bash you"
)

# ✅ Correct: pass the whole string to the tokenizer
token_lists = [tokenizer(song)]

# Build vocabulary
vocab = build_vocab_from_iterator(token_lists, specials=["<unk>"])
vocab.set_default_index(vocab["<unk>"])

# Create pipeline
text_pipeline = lambda x: vocab(tokenizer(x))

# Inspect vocab
index_to_token = vocab.get_itos()
print(index_to_token[:20])



['<unk>', 'gonna', 'never', 'you', 'and', 'the', 'your', "'", 'break', 'it', 'loop', 'of', 'to', 'a', 'are', 'ask', 'backprop', 'bash', 'batch', 'code']


In [59]:
print(tokenizer('never going to give you up'))

['never', 'going', 'to', 'give', 'you', 'up']


In [60]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator
from torch.utils.data import DataLoader, Dataset
import tqdm

In [61]:
def train(dataloader, model, number_of_epochs=100, show=10):
    MY_LOSS = []
    for epoch in tqdm(range(number_of_epochs)):
        total_loss = 0 

        for context, target in dataloader:
            model.zero_grad()
            predicted = model(context)
            loss.backward()
            optimizer.step()
            loss = criterion(predicted, target.reshape(-1))
            total_loss += loss.item()

            MY_LOSS.append(total_loss/len(dataloader))
    return MY_LOSS

In [62]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")


padding=BATCH_SIZE-len(tokens)%BATCH_SIZE

tokens_pad=tokens+tokens[0:padding]
dataloader = DataLoader(tokens_pad, batch_size=BATCH_SIZE, shuffle=False, collate_fn=collate_batch)

criterion = torch.nn.CrossEntropyLoss()

model = NgramLanguageModeler(len(vocab), EMBEDDING_DIM, CONTEXT_SIZE).to(device)

optimizer = optim.SGD(model.parameters(), lr=0.01)

Using device: cpu


In [63]:
index_to_token = vocab.get_itos()
print(index_to_token)

['<unk>', 'gonna', 'never', 'you', 'and', 'the', 'your', "'", 'break', 'it', 'loop', 'of', 'to', 'a', 'are', 'ask', 'backprop', 'bash', 'batch', 'code', 'crash', 'dancing', 'don’t', 'drop', 'every', 'explode', 'feels', 'give', 'gradients', 'how', 'i', 'if', 'it’s', 'i’ll', 'just', 'layers', 'lie', 'loss', 'math', 'matrix', 'me', 'ml', 'model', 'night', 'no', 'out', 'overfit', 'parody', 'ram', 'right', 'run', 'running', 's', 'say', 'strangers', 'tell', 'tensors', 'those', 'through', 'til', 'train', 'tune', 'tuning', 'undershoot', 'up', 'watch', 'way', 'we', 'weights', 'with', 'works', 'worry']


In [64]:
context = text_pipeline("Never gonna")


x_c = torch.tensor(context).reshape(-1,1)

out=model(x_c)
predicted_index = torch.argmax(out,1)

print(predicted_index)

display(index_to_token[predicted_index])


tensor([69])


'with'

In [65]:
def write_song(model, number_of_words=100):
    my_song=""
    for i in range(number_of_words):
        with torch.no_grad():
            context =torch.tensor(vocab([tokens[i-j-1-1] for j in range(CONTEXT_SIZE)])).to(device)
            word_inx = torch.argmax(model(context))
            my_song+=" "+index_to_token[word_inx.detach().item()]
    return my_song

In [71]:
print(write_song(model, number_of_words=100))


 loop loop those s loop ask i’ll loop loop loop run with with with s those ask loop <unk> ask with s loop run watch layers give i’ll ask ask loop matrix those ask <unk> those drop those loop loop loop drop matrix feels loop give ask loop ask explode crash weights explode ask you i’ll layers break loop loop loop drop you run loop explode give loop loop loop ask <unk> bash give s loop ask i’ll ask s loop loop if every s <unk> run are loop explode break loop those s drop with s bash ask s
